# **Data-Science Fundamentals Final Project: predicting flights using ML algorithms**

## Stage No. 1: collecting the data:
- for this stage we will use a web scraper-collector that will collect flights data from two websites: Kiwi and Kayak.
- this collector is built with async-runtime functions, random user-actions generator, session saver, cookies saver, dynamic viewport and even DHCP-ip-refresher function (since were not using proxy), all of these methods are used for collecting without getting cought by these websities bot-identifier machenisms...
- this collector will run every combination of ttt (time to travel) from 1-30, los (lenght of stay) from 1-5 within a 3 different snapshot days for every combination of rome,london and paris routes

In [ ]:
from scraper.flights_scraping import get_data

get_data()



after we have collected the data: were doing the usual imports:

In [16]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

Exploring the data

In [17]:
data = pd.read_csv('Flights.csv')
data.head()

,departure_hour,departure_airport,flight_length,landing_hour,landing_airport,to_dest_company,return_departure_hour,return_departure_airport,return_flight_length,return_landing_hour,...,ttt,los,snapshot_date,origin_city,destination_city,departure_date,return_date,website,layover_time,return_layover_time
0,6:00a,CIA,2h 45m,7:45a,STN,Ryanair,6:10p,STN,2h 25m,9:35p,...,2,1,2025-02-28,ROME,LONDON,2025-03-02,2025-03-03,Kayak,0m,0m
1,6:35a,FCO,2h 45m,8:20a,LGW,Wizz Air,9:10a,STN,2h 30m,12:40p,...,2,1,2025-02-28,ROME,LONDON,2025-03-02,2025-03-03,Kayak,0m,0m
2,6:35a,FCO,2h 45m,8:20a,LGW,Wizz Air,6:10p,STN,2h 25m,9:35p,...,2,1,2025-02-28,ROME,LONDON,2025-03-02,2025-03-03,Kayak,0m,0m
3,6:00a,CIA,2h 45m,7:45a,STN,Ryanair,6:10p,STN,2h 25m,9:35p,...,2,1,2025-02-28,ROME,LONDON,2025-03-02,2025-03-03,Kayak,0m,0m
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,1,2025-02-28,ROME,LONDON,2025-03-02,2025-03-03,Kayak,0m,0m


In [18]:
data.tail()

,departure_hour,departure_airport,flight_length,landing_hour,landing_airport,to_dest_company,return_departure_hour,return_departure_airport,return_flight_length,return_landing_hour,...,ttt,los,snapshot_date,origin_city,destination_city,departure_date,return_date,website,layover_time,return_layover_time
493306,07:00,CDG,1h 20m,07:20,LTN,easyJet,13:25,15:45,1h 20m,15:45,...,14,5,2025-03-02,PARIS,LONDON,2025-03-16,2025-03-21,Kiwi,0m,0m
493307,07:00,ORY,1h 10m,07:10,LGW,Vueling,14:20,16:30,1h 10m,16:30,...,14,5,2025-03-02,PARIS,LONDON,2025-03-16,2025-03-21,Kiwi,0m,0m
493308,09:55,CDG,1h 20m,10:15,LTN,easyJet,20:15,22:30,1h 15m,22:30,...,14,5,2025-03-02,PARIS,LONDON,2025-03-16,2025-03-21,Kiwi,0m,0m
493309,07:00,CDG,1h 20m,07:20,LTN,easyJet,19:35,21:55,1h 20m,21:55,...,14,5,2025-03-02,PARIS,LONDON,2025-03-16,2025-03-21,Kiwi,0m,0m
493310,07:00,ORY,1h 10m,07:10,LGW,Vueling,15:15,17:45,1h 30m,17:45,...,14,5,2025-03-02,PARIS,LONDON,2025-03-16,2025-03-21,Kiwi,0m,0m


In [22]:
data.columns

Index(['departure_hour', 'departure_airport', 'flight_length', 'landing_hour',
       'landing_airport', 'to_dest_company', 'return_departure_hour',
       'return_departure_airport', 'return_flight_length',
       'return_landing_hour', 'return_landing_airport', 'return_company',
       'price', 'layover', 'ttt', 'los', 'snapshot_date', 'origin_city',
       'destination_city', 'departure_date', 'return_date', 'website',
       'layover_time', 'return_layover_time'],
      dtype='object')

In [19]:
data.describe()

,ttt,los
count,493311.000000,493311.000000
mean,13.221976,3.031453
std,7.755038,1.415066
min,1.000000,1.000000
25%,7.000000,2.000000
50%,13.000000,3.000000
75%,19.000000,4.000000
max,30.000000,5.000000


the just 2 numeric features

we will add a function that will convert the time strings to int

In [9]:
import re
def time_to_minutes(time_str: str)-> int:
    if pd.isna(time_str):
        return 0

    # Extract hours and minutes using regex
    hours = 0
    minutes = 0
    
    h_match = re.search(r'(\d+)h', time_str)
    m_match = re.search(r'(\d+)m', time_str)

    if h_match:
        hours = int(h_match.group(1))
    if m_match:
        minutes = int(m_match.group(1))

    return hours * 60 + minutes


In [20]:
data['flight_length'] = data['flight_length'].apply(time_to_minutes)
data['return_flight_length'] = data['return_flight_length'].apply(time_to_minutes)
data['layover_time'] = data['layover_time'].apply(time_to_minutes)
data['return_layover_time'] = data['return_layover_time'].apply(time_to_minutes)

In [21]:
data.describe()

,flight_length,return_flight_length,ttt,los,layover_time,return_layover_time
count,493311.000000,493311.000000,493311.000000,493311.000000,493311.000000,493311.000000
mean,124.434588,121.223607,13.221976,3.031453,6.431229,5.788608
std,67.111854,63.021224,7.755038,1.415066,44.118978,39.975479
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,80.000000,80.000000,7.000000,2.000000,0.000000,0.000000
50%,130.000000,125.000000,13.000000,3.000000,0.000000,0.000000
75%,150.000000,145.000000,19.000000,4.000000,0.000000,0.000000
max,1635.000000,1635.000000,30.000000,5.000000,1495.000000,1495.000000


lets convert the hours to int format, and remove dummy vars